
# Base de Dados

</br>

<img src="image/python_supermercado_2025.jpg" alt="Descrição da imagem" width="80%">

O objetivo deste notebook é realizar a leitura e estruturação da base de dados que será utilizada para a modelagem. A base de dados contém notas fiscais de dois supermercados atacadistas, cobrindo o período de junho de 2022 a junho de 2025.


<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">


## 1. Configura as bibliotecas

In [0]:
# !pip install nfceget
# !pip install bs4
# !pip install openpyxl

In [0]:
import time
import pandas as pd
from functools import reduce
from nfceget import app
import hashlib
import openpyxl
from utils.tic_toc import tictoc

## 2. Define os parâmetros

Aqui estão os parâmetros que deverão ser alterados conforme necessidade.

In [0]:
tic_geral = time.time()
tic = time.time()

dir = '/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket'

input_file = f'{dir}/data/qrcodes.txt'
de_para_file = f'{dir}/data/de_para_nfce.xlsx'
output_file = f'{dir}/data/notas_fiscais_supermercado.xlsx'

toc = time.time()
print(f'\n\033[33mEtapa 2 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 3. Leitura da base de dados

A base de dados consiste em um arquivo TXT que armazena os códigos QR das minhas notas fiscais de supermercado, fornecidas pela Secretaria de Estado da Fazenda do Paraná. Existe um pacote Python que é capaz de extrair informações da página web acessada através do link do código QR em cada nota fiscal do consumidor. Este pacote será empregado para ler essas informações.


In [0]:
tic = time.time()

with open(input_file, 'r') as arquivo:
    arquivo = arquivo.readlines()

qrcodes = []
for linha in arquivo:
    qrcodes.append(linha.strip()) # Não deixe linhas com espaço em branco no arquivo.

dicionarios = []
for dic in qrcodes:
    dicionarios.append(app.json_from_qrcode_link(dic))

dataframes = []
for nota in dicionarios:
    df_nota = pd.DataFrame(nota['itens'])

    for key, value in nota['local'].items():
        df_nota['local_' + key] = value
    for key, value in nota['nfce'].items():
        df_nota['nfce_' + key] = value

    dataframes.append(df_nota)

df = pd.concat(dataframes, ignore_index=True)

df[['name', 'code', 'quantity', 'unitaryValue', 'totalValue', 'local_name', 'local_cnpj', 'nfce_date']].display()

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 3 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 4. Anonimação de dados sensíveis

Para evitar divulgar dados sensíveis, não irei disponibilizar o arquivo txt dos códigos QR, apenas o arquivo final em xlsx já tratado. Caso deseje replicar o código com as suas notas fiscais basta criar um arquivo txt com os links do código QR disponível na nota, que ficam no formato:

- https://www.fazenda.pr.gov.br/nfce/qrcode?p=1234567890ABC1234567890ABC1234567890ABC1234567890ABC1234567890ABC1234567890ABC1234567890ABC12345678 

In [0]:
tic = time.time()

# Criar mapeamento original -> hash
def hash_nfce(chave):
    return hashlib.sha256(chave.encode()).hexdigest()

df['nfce_chave_anon'] = df['nfce_chave'].astype(str).apply(hash_nfce)
de_para = df[['nfce_chave', 'nfce_chave_anon']].drop_duplicates()

toc = time.time()
print(f'\n\033[33mEtapa 4 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 5. Preparação dos dados

In [0]:
tic = time.time()

### 5.1 Deletar colunas

Para facilitar a manipulação de dados, vou selecionar apenas as variáveis que serão utilizadas de alguma forma e renomear com um nome mais sujestivo.

- name: PRODUTO	
- code:	COD_PRODUTO
- quantity: QTDE
- unit: UNIDADE
- unitaryValue: VALOR_UNIDADE
- totalValue: VALOR_TOTAL
- local_name: <font color="red">Será deletada!</font> 
- local_cnpj: CNPJ
- local_address: <font color="red">Será deletada!</font>
- nfce_numero: <font color="red">Será deletada!</font>
- nfce_serie: <font color="red">Será deletada!</font>
- nfce_date: DATA
- nfce_protocolo: <font color="red">Será deletada!</font>
- nfce_version: <font color="red">Será deletada!</font>
- nfce_chave: <font color="red">Será deletada!</font>
- nfce_chave_anon: CHAVE

In [0]:
# Deletar colunas
df = df.drop(columns=['local_name', 
                      'local_address',
                      'nfce_numero', 
                      'nfce_serie', 
                      'nfce_protocolo', 
                      'nfce_version',
                      'nfce_chave'])

# Renomear colunas
df = df.rename(columns={
    'name': 'PRODUTO',
    'code': 'COD_PRODUTO',
    'quantity': 'QTDE',
    'unit': 'UNIDADE',
    'unitaryValue': 'VALOR_UNIDADE',
    'totalValue': 'VALOR_TOTAL',
    'local_cnpj': 'CNPJ',
    'nfce_date': 'DATA',
    'nfce_chave_anon': 'CHAVE'
})

### 5.2 Filtrar dados

A base de dados contém informações de várias notas fiscais, o foco do estudo são compras em supermercados, sendo assim vou selecionar os dois CNPJs dos supermercados que mais frequento, evitando assim que misture notas fiscais de outros produtos que não seja supermercado. 

- IRMAOS MUFFATO S A: 76.430.438/0070-01 (MAX)
- SENDAS DISTRIBUIDORA S/A: 06.057.223/0367-96 (ASSAI)

In [0]:
# Mantendo apenas o MAX e o ASSAI:
cnpjs = ['76.430.438/0070-01', '06.057.223/0367-96']
df = df[df['CNPJ'].isin(cnpjs)]

# Coluna 'SUPERMERCADO'
df['SUPERMERCADO'] = df['CNPJ'].apply(lambda x: 'MAX' if x == '76.430.438/0070-01' else ('ASSAI' if x == '06.057.223/0367-96' else x))

# Deletar a coluna 'CNPJ'
df = df.drop(columns=['CNPJ'])

# Dimensão:
num_linhas, num_colunas = df.shape
print(f"""
- Número de linhas: \033[93m{num_linhas:,}\033[0m 
- Número de colunas: \033[93m{num_colunas:,}\033[0m""".replace(',', '.'))

### 5.3 Sintetizar informações

É possível que o operador de caixa registre o mesmo produto em sequências diferentes. Isso pode resultar na aparição repetida de um produto na mesma nota fiscal, sem que as quantidades sejam somadas corretamente.

Para contornar esse problema, a base de dados será organizada por produto para cada chave única. O valor unitário será mantido, enquanto as quantidades e os valores totais serão somados.

In [0]:
# Ajustando os tipos para fazer operações
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y %H:%M:%S')

for col in ['VALOR_UNIDADE', 'QTDE', 'VALOR_TOTAL']:
    df[col] = df[col].str.replace(',', '.').astype(float)
df[['VALOR_UNIDADE', 'QTDE', 'VALOR_TOTAL']] = df[['VALOR_UNIDADE', 'QTDE', 'VALOR_TOTAL']].round(2)

# Agrupando:
df = df.groupby(['CHAVE', 'DATA', 'COD_PRODUTO', 'PRODUTO']).agg({
    'SUPERMERCADO': 'first',  # Mantém o nome do supermercado
    'UNIDADE': 'first',  # Mantém a unidade
    'VALOR_UNIDADE': 'first',  # Mantém o valor unitário
    'QTDE': 'sum',  # Soma a quantidade
    'VALOR_TOTAL': 'sum'  # Soma o valor total
}).reset_index()

# Dimensão:
num_linhas, num_colunas = df.shape
print(f"""
- Número de linhas: \033[93m{num_linhas:,}\033[0m 
- Número de colunas: \033[93m{num_colunas:,}\033[0m""".replace(',', '.'))

### 5.4 Criar variáveis

A variável data contém informações sobre o dia, mês, ano, hora, minuto e segundo. Vou criar uma nova variável chamada periodo para representar a parte do dia (manhã, tarde ou noite) com base nos dados da variável data. Além disso, será criado também uma variável que define as categorias dos produtos de acordo com palavras chave no nome do produto.

In [0]:
# Período do dia
df['PERIODO'] = pd.cut(df['DATA'].dt.hour, bins=[0, 12, 18, 24], labels=['Manhã', 'Tarde', 'Noite'], include_lowest=True)

# Categoria de produtos
categoria_produtos = {
    'mercearia': [
        'ARROZ', 'FEIJAO', 'FARINHA', 'TRIGO', 'ARROZ INTEGRAL', 'FEIJAO PRETO', 
        'FEIJ', 'ACUCAR', 'ACUC', 'SUGAR', 'OVO', 'OLEO', 'COAMO', 'FAROFA', 'YOKI',
        'ARR', 'FARO'
    ],
    'massa': [
        'MAC', 'MASSA', 'LASANHA', 'NISSIN', 'ESPAGUETE', 'PENNE', 'PINDUCA', 
        'TALHARIM', 'FETTUCCINE', 'GNOCCHI', 'RAVIOLES', 'MAC INTEGRAL', 'CONCHINHA', 
        'MAC GRANO', 'MASSA FRESCA', 'LAS PERD'
    ],
    'cereal': [
        'CEREAL', 'NESTLE', 'NESCAU', 'SUCRILHOS', 'AVEIA', 'FLOCOS', 'KELLOGGS', 
        'GRANOLA', 'MUESLI', 'BARRA', 'CEREAIS INTEGRAL', 'FLOCOS MILHO', 'CEREAL MAT', 
        'BARRA DE CEREAL', 'AVEIA INSTANT', 'SNOW FLAKES'
    ],
    'bebida': [
        'BEB', 'FANTA', 'AG COC', 'COCA', 'SUCO', 'REFRI', 'ENERGETICO', 'AGUA', 
        'REFRIG', 'GATORADE', 'ADES', 'CRYSTAL', 'MONSTER', 'DELV', 'NATURAL O', 
        'LIFE', 'SUCOS', 'TEA', 'VITAMINA', 'MAIS', 'SOYA', 'TANG', 'GUARANA', 
        'CAFE', 'CHA', 'LEITE', 'SUCO DE FRUTA', 'REFRIG LITE', 'ENERG', 'PRATSY',
        'REF', 'SCHWEPPES', 'SCHWEP', 'CITRUS', 'DEL VALLE', 'VALLE', 'YOPRO',
        'PURITY', 'GELADINH', 'CRISTAL'
    ],
    'bebida alcoolica': [
        'VODKA', 'CHAMPAGNE', 'CERVEJA', 'SKOL', 'HEINEKEN', 
        'BUDWEISER', 'SPATEN', 'PETRA', 'BRAHMA', 'STELLA', 'ESTRELLA', 'VINHO', 
        'ICE', 'PERIQUIT', 'SMIRNOFF', 'BEATS', 'PASSPORT', 'WHISKY', 'COROT', 
        'CAMPO', 'SUAVE', 'GIN', 'SAKE', 'TEQUILA', 'CERVEJA ARTESANAL', 'VINHO TINTO', 
        'VINHO BRANCO', 'WHISK', 'JACK DANIEL', 'CERV', 
    ],
    'padaria e frios': [
        'MORTADELA', 'PRESUNTO', 'QUEIJO', 'SALAME', 'PEPPA', 'MORTADELA FRANGO', 
        'FRIO VARIADOS', 'QUEIJO RALADO', 'PAO', 'BOLINHO', 'WAFER', 'BOLO', 'MUFFIN', 
        'BAUDUCCO', 'MARILAN', 'VISCONTI', 'BAGUETE', 'PAO INTEGRAL', 'PAO QUEIJO', 
        'PAO DOCE', 'PAO FRANCES', 'BOLO RECHEADO', 'PRES', 'AURORA', 'P FORMA', 'QJO',
        'MUSS', 'MORT'
    ],
    'congelado': [
        'PIZZA', 'SALSICHA', 'SORVETE', 'GELADO', 'CONGELADO', 'LASANHA', 'EMPANADO', 
        'SEARA', 'FRIMESA', 'ICE CREAM', 'BATATA FRITA', 'FRUTAS CONGELADAS', 
        'REFEICAO PRONTA', 'PEIXE CONGELADO', 'SALSICHA CONGELADA', 'TEKITOS', 'PERDIG',
        'SALS'
    ],
    'acougue': [
        'FRANGO', 'PEITO', 'FILE', 'BIFES', 'CARNES', 'BACON', 'CORTES', 'CARNE SUINA', 
        'COXA', 'LINGUICA', 'PRESUNTO', 'SALSICHA', 'PERDIGAO', 'SEARA', 'FRIMESA', 
        'CARNE BOV', 'ALCATRA', 'CONTRA FILE', 'COSTELA', 'COSTELA SUINA', 'CARNE MOIDA',
        'BIF ACEBOLA', 'MEIO ASA', 'SADIA', 'FRALDINHA', 'COX MOLE', 'FRIBOI', 'FRIB',
        'ACEM MO'
    ],
    'biscoito e snack': [
        'BISCOITO', 'CHIPS', 'SNACK', 'FANDANGOS', 'CHEETOS', 'PRINGLES', 'WAFER', 
        'BOLACHA', 'BOLINHO', 'BISCOITOS INTEGRAL', 'SNACK SALG', 'BATATA PALHA', 
        'BOLINHO ARROZ', 'SNACK DOCE', 'SALGADOS', 'ELMA', 'SALG', 'PALHA', 'TORR',
        'BISC'
    ],
    'bomboniere': [
        'BALA', 'CHOCOLATE', 'BOMBOM', 'KINDER', 'RAFFAELLO', 'TRIDENT', 
        'HALLS', 'DORI', 'BALAS GOMA', 'PIPOCA DOCE', 'CONFETE', 'CHOC BRANCO', 'LACTA',
        'NUTELLA', 'PIRULITO', 'GELATINA'
    ],
    'laticinio': [
        'LEITE', 'QUEIJO', 'IOGURTE', 'CREME', 'MANTEIGA', 'REQUEIJAO', 'ACHOCOLATADO', 
        'NATURAL', 'PRESIDENT', 'TIROL', 'BATAVO', 'ELEGE', 'MUSSARELA', 'CREME LEITE', 
        'REFRIGERADO', 'UHT', 'REQUEIJAO CREMOSO', 'QUEIJO CREMOSO', 'IOGUR', 'CREM LTE',
        'CREM LEI', 'LEITE COND', 'CR LEI', 'ACHOC', 'CHOCO MILK', 'IOG', 'LIDER', 'DANONINHO',
        'CHOC', 'CHOCOMILK', 'BATAV', 'MOOCA'
    ],
    'molho': [
        'MOLHO', 'KETCHUP', 'CATCHUP', 'MAIONESE', 'TEMPERO', 'MOSTARDA', 'SAZON', 
        'COND', 'HEMMER', 'HELLMANN', 'QUERO', 'POMAROLA', 'PICKLES', 'CREME CEBOLA', 
        'SALSA', 'MOLHO TARATAR', 'MOLHO PIMENTA', 'ELEFANT', 'CATCH', 'ELEF', 'MAION',
        'MOST', 'HELLM', 'HELL'
    ],
    'lataria e conserva': [
        'MILHO', 'ERVILHA', 'LATA', 'CONSERVA', 'SARDINHA', 'ATUM', 'PALMITO', 
        'EXTRATO', 'BONARE', 'FUGINI', 'SELETA', 'AZEITONA', 'BERINJELA', 
        'MILHO EM CONSERVA', 'ERVILHA EM LATA'
    ],
    'condimento': [
        'ALHO', 'SAZON', 'TEMPERO', 'TEMP', 'PIMENTA', 'AZEITE', 'MANJER', 'CHIMICHURRI',
        'SAL', 'KITANO', 'KININO', 'LOURO', 'CHIMICH', 'CHEIRO', 'TRIANGULO', 'OREGANO'
    ],
    'limpeza e limpeza': [
        'LIMP', 'DETERG', 'SABAO', 'OMO', 'LYSOL', 'VEJA', 'DESINF', 'PINHO', 'SANIT', 
        'AJAX', 'UOL', 'ALVEJANTE', 'MULTIUSO', 'ESPONJA', 'LIMP', 'DESENGORD', 
        'LIMP PERFUMADO', 'PILHA', 'PINO', 'EXTENS', 'AMIDO', 'TOALHA', 'ESCOVA',
        'FIO DENT', 'SAB DOVE', 'PAP H', 'SHAMPOO', 'COND', 'DESOD', 'SABONETE',
        'NEVE', 'FRALDA', 'CREME DENTAL', 'HEAD SHOULDERS', 'NIVEA', 'PAMPERS',
        'PROT', 'DESODOR', 'CREME HIDR', 'LENCO', 'COTONETE', 'LEN DESCART',
        'FIO DENTAL', 'COLGATE', 'SBP', 'REPEL', 'ABOVE', 'DENT', 'SORRISO', 'DOWN',
        'COLG', 'QBOA', 'QUIBOA', 'ESC', 'BABY', 'SOFT', 'DIABO', 'SBT', 'YPE', 'PURIF'
    ],
    'hortifruti': [
        'CEBOLA', 'TOMATE', 'BANANA', 'CENOURA', 'ALFACE', 'MANDIOCA', 'UVA', 'MACA', 
        'BATATA', 'LARANJA', 'SFERRIE', 'VERDE', 'CEBOLINHA', 'SALSINHA', 'HORTELA',
        'ESPINAFRE', 'ABOBORA', 'ABOBRINHA', 'MORANGO', 'MANGA', 'RUCULA', 
        'BATATA DOCE', 'BETERRABA', 'BROCOLIS', 'PIMENTAO', 'FRUTAS VARIADAS',
        'LIMAO', 'MGO', 'LARAN'
    ],
    'cosmetico': [
        'CREME', 'NIELY', 'SHAMPOO', 'SABONETE', 'VENUS', 'NIVEA', 'LOCAO', 'DOVE', 
        'NEUTROGENA', 'BATOM', 'MAQUIAGEM', 'ESMALTE', 'RIMEL', 'PO', 'BASE', 'ILUM', 
        'CREME FACIAL', 'LOCAO HIDR', 'SHOUL', 'P H MAX'
    ],
    'utilidades': [
        'COPO', 'PRATO', 'GARFO', 'EXTENS', 'PINO', 'SACO', 'ROLO', 'LIXEIRA', 
        'PAPEL ALUM', 'FILME PVC', 'PLASVALE', 'SACO LIXO', 'EMBAL A VACUO', 
        'CAIXA', 'EMBAL PLAST', 'SACO DESCART', 'PARANA', 'FRIGD', 'VELA', 'FILM',
        'MINI FRIGID', 'PANELA', 'COPAC', 'CESTO', 'CARVAO', 'PANO', 'LAMPADA', 'RODO',
        'FILT', 'DA ILHA', 'ESP'
    ],
    'pet': [
        'RACAO', 'PETISCO', 'COLEIRA', 'BRINQUEDO', 'AREIA', 'GATO', 'CACHORRO', 
        'WHISKAS', 'PEDIGREE', 'PURINA', 'CAMA', 'TRANSPORTADORA', 'RACAO HUMIDA', 
        'PETISCOS VARIADOS'
    ],
    'outros': []
}

# Função para classificar produtos
def classificar_produto(nome_produto):
    for categoria, palavras_chave in categoria_produtos.items():
        for palavra in palavras_chave:
            if palavra in nome_produto.upper():
                return categoria.upper()
    return 'OUTROS'

df['CATEGORIA_PRODUTO'] = df['PRODUTO'].apply(classificar_produto)

# Dimensão:
num_linhas, num_colunas = df.shape
print(f"""
- Número de linhas: \033[93m{num_linhas:,}\033[0m 
- Número de colunas: \033[93m{num_colunas:,}\033[0m""".replace(',', '.'))

### 5.5 Formatar datas

Formatarei a data para o padrão dd/mm/aaaa.

In [0]:
df['DATA'] = df['DATA'].dt.strftime('%d/%m/%Y')

display(df)

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 5 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 6. Salva em Excel

In [0]:
tic = time.time()

de_para.to_excel(de_para_file, index=False)
df.to_excel(output_file, index=False)

toc = time.time()
print(f'\n\033[33mEtapa 6 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 7. Tempo decorrido

In [0]:
toc_geral = time.time()
print(f'\n\033[33mTempo decorrido no notebook:\033[0;0m {tictoc(tic_geral, toc_geral)}')